# Notebook contains summary of adversarial robustness of different model

In [1]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
d = os.path.dirname(os.getcwd())
sys.path.insert(0, d)

In [2]:
from datetime import datetime as dt
import torch
print(torch.__version__)
print(torch.cuda.is_available())

import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import torchvision
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms

from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from utils.attacks import fgsm, pgd
from utils.utils import plot_images
from models.resnet import ResNet, SparseResNet
from models.wide_resnet import WideResNet, SparseWideResNet

1.7.0
True


In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [4]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
])

ds = CIFAR10('../data', train=False, target_transform=None, download=True, transform=test_transform)
test_loader = DataLoader(ds, 400, shuffle=True)

Files already downloaded and verified


In [5]:
def clean_acc(model, loader):
    clean_correct = 0
    for k, (X, y) in enumerate(loader, 1):
        start = dt.now().replace(microsecond=0)
        X, y = X.to(device), y.to(device)  
        clean_correct += (model(X).max(dim=1)[1] == y).sum().item()
        end = dt.now().replace(microsecond=0)
        print(f"iter: {k}/{len(loader)} time: {end-start}", end="\r")
    print(" " * 50, end="\r")
    return round(clean_correct / len(loader.dataset), 4)

def adv_acc(model, loader, loss_fn, attack_method, attack_params):
    adv_correct = [0] * len(attack_params)
    for k, (X, y) in enumerate(loader, 1):
        start = dt.now().replace(microsecond=0)
        X, y = X.to(device), y.to(device)
        for i, p in enumerate(attack_params):
            noise = attack_method(model, X, y, loss_fn, epsilon=p["epsilon"], 
                                  alpha=p["alpha"], num_iter=p["num_iter"])
            adv_correct[i] += (model(X+noise).max(dim=1)[1] == y).sum().item()
        end = dt.now().replace(microsecond=0)
        print(f"iter: {k}/{len(loader)} time: {end-start}", end="\r")
    print(" " * 50, end="\r")
    return [round(a/len(loader.dataset), 4) for a in adv_correct]

In [6]:
attack1 = {
    "id": 0,
    "name": "eps=8,a=2,i=20",
    "epsilon": 8/255,
    "alpha": 2/255,
    "num_iter": 20
}

attack2 = {
    "id": 1,
    "name": "eps=4,a=2,i=20",
    "epsilon": 4/255,
    "alpha": 2/255,
    "num_iter": 20
}


attack3 = {
    "id": 0,
    "name": "eps=8,a=2,i=40",
    "epsilon": 8/255,
    "alpha": 2/255,
    "num_iter": 40
}

attack4 = {
    "id": 1,
    "name": "eps=4,a=2,i=40",
    "epsilon": 4/255,
    "alpha": 2/255,
    "num_iter": 40
}

params = [attack1, attack2]
params2 = [attack3, attack4]
pd_clean, pd_adv = [], []

In [8]:
model = WideResNet(depth=28, widen_factor=10, dropout_rate=0.3, num_classes=10).to(device)
model.load_state_dict(torch.load("../saved/wide_resnet.pt", map_location=device))

<All keys matched successfully>

In [9]:
acc_clean = clean_acc(model, test_loader)
pd_clean.append(acc_clean)
print(f"clean accuracy: {acc_clean}")

acc_adver = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params)
pd_adv.append(acc_adver)
for a, p in zip(acc_adver, params):
    print(f"{p['name']}: {a}")

clean accuracy: 0.9268                            
eps=8,a=2,i=20: 0.0012                            
eps=4,a=2,i=20: 0.0789


**SparseWideResNet**

In [10]:
model = SparseWideResNet(depth=28, widen_factor=10, k_winners=0.1, num_classes=10).to(device)
model.load_state_dict(torch.load("../saved/sparse_wide_resnet.pt", map_location=device))

<All keys matched successfully>

In [11]:
acc_clean = clean_acc(model, test_loader)
pd_clean.append(acc_clean)
print(f"clean accuracy: {acc_clean}")

acc_adver = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params)
pd_adv.append(acc_adver)
for a, p in zip(acc_adver, params):
    print(f"{p['name']}: {a}")

clean accuracy: 0.9116                            
eps=8,a=2,i=20: 0.4014                            
eps=4,a=2,i=20: 0.5987


In [12]:
d = {
    'model': ["Wide-ResNet", "Sparse Wide-ResNet"], 
    'clean img accuracy': pd_clean, 
}
pd_adv = np.array(pd_adv)
for p in params:
    d[p["name"]] = pd_adv[:, p["id"]]
    
df = pd.DataFrame(data=d)
df

,model,clean img accuracy,"eps=8,a=2,i=20","eps=4,a=2,i=20"
0,Wide-ResNet,0.9268,0.0012,0.0789
1,Sparse Wide-ResNet,0.9116,0.4014,0.5987


### Parameter k in K-Winners

In [25]:
pd_clean, pd_adv = [], []

In [15]:
model = SparseWideResNet(depth=28, widen_factor=10, k_winners=0.2, num_classes=10).to(device)
model.load_state_dict(torch.load("../saved/sparse_wide_resnet_02.pt", map_location=device))

<All keys matched successfully>

In [ ]:
acc_clean = clean_acc(model, test_loader)
pd_clean.append(acc_clean)
print(f"clean accuracy: {acc_clean}")

acc_adver = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params)
pd_adv.append(acc_adver)
for a, p in zip(acc_adver, params):
    print(f"{p['name']}: {a}")

In [27]:
model = SparseWideResNet(depth=28, widen_factor=10, k_winners=0.05, num_classes=10).to(device)
model.load_state_dict(torch.load("../saved/sparse_wide_resnet_005.pt", map_location=device))

<All keys matched successfully>

In [28]:
acc_clean = clean_acc(model, test_loader)
pd_clean.append(acc_clean)
print(f"clean accuracy: {acc_clean}")

acc_adver = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params)
pd_adv.append(acc_adver)
for a, p in zip(acc_adver, params):
    print(f"{p['name']}: {a}")

clean accuracy: 0.884                             
eps=8,a=2,i=20: 0.548                             
eps=4,a=2,i=20: 0.6596


In [29]:
d = {
    'model': ["Sparse Wide-ResNet, k=0.1", "Sparse Wide-ResNet k=0.2", "Sparse Wide-ResNet k=0.05"], 
    'clean img accuracy': pd_clean, 
}
pd_adv = np.array(pd_adv)
for p in params:
    d[p["name"]] = pd_adv[:, p["id"]]
    
df = pd.DataFrame(data=d)
df

,model,clean img accuracy,"eps=8,a=2,i=20","eps=4,a=2,i=20"
0,"Sparse Wide-ResNet, k=0.1",0.9116,0.4014,0.5987
1,Sparse Wide-ResNet k=0.2,0.9186,0.0398,0.3137
2,Sparse Wide-ResNet k=0.05,0.8840,0.5480,0.6596


In [9]:
model = SparseWideResNet(depth=28, widen_factor=10, k_winners=0.1, num_classes=10).to(device)
model.load_state_dict(torch.load("../saved/sparse_wide_resnet.pt", map_location=device))

<All keys matched successfully>

In [10]:
acc_clean = clean_acc(model, test_loader)
pd_clean.append(acc_clean)
print(f"clean accuracy: {acc_clean}")

params2 = [attack3]

acc_adver = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params2)
pd_adv.append(acc_adver)
for a, p in zip(acc_adver, params2):
    print(f"{p['name']}: {a}")

clean accuracy: 0.9117                            
eps=8,a=2,i=40: 0.364                             


## Adversarial training

In [7]:
pd_clean, pd_adv = [], []

In [8]:
model = SparseWideResNet(depth=28, widen_factor=10, k_winners=0.1, num_classes=10).to(device)
model.load_state_dict(torch.load("../saved/robust_sparse_wide_resnet.pt", map_location=device))

<All keys matched successfully>

In [9]:
acc_clean = clean_acc(model, test_loader)
pd_clean.append(acc_clean)
print(f"clean accuracy: {acc_clean}")

acc_adver = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params)
pd_adv.append(acc_adver)
for a, p in zip(acc_adver, params):
    print(f"{p['name']}: {a}")

clean accuracy: 0.7878                            
eps=8,a=2,i=20: 0.8286                            
eps=4,a=2,i=20: 0.7781


In [10]:
d = {
    'model': ["Robust Sparse Wide-ResNet, k=0.1"], 
    'clean img accuracy': pd_clean, 
}
pd_adv = np.array(pd_adv)
for p in params:
    d[p["name"]] = pd_adv[:, p["id"]]
    
df = pd.DataFrame(data=d)
df

,model,clean img accuracy,"eps=8,a=2,i=20","eps=4,a=2,i=20"
0,"Robust Sparse Wide-ResNet, k=0.1",0.7878,0.8286,0.7781
